In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import graph_tool.all as gt
import networkx as nx
import numpy as np

from llm_ol.dataset import data_model
from llm_ol.utils.nx_to_gt import nx_to_gt

In [ ]:
# G = data_model.load_graph("out/data/wikipedia/v1/full/graph_depth_3.json")
# G = data_model.load_graph("out/experiments/prompting/dev-h-v2/graph.json")
G = data_model.load_graph("out/experiments/hearst/v1/graph.json")

In [ ]:
G_gt = nx_to_gt(G)

In [ ]:
motifs, counts, vertex_maps = gt.motifs(G_gt, 3, return_maps=True)
# order = np.argsort(counts)[::-1]
# motifs = [motifs[i] for i in order]
# counts = [counts[i] for i in order]
# vertex_maps = [vertex_maps[i] for i in order]

# motifs, counts = gt.motifs(G_gt, 4, return_maps=False)
# order = np.argsort(counts)[::-1]
# motifs = [motifs[i] for i in order]
# counts = [counts[i] for i in order]
print(counts)

In [ ]:
sub_G = gt.GraphView(G_gt, vfilt=lambda v: v in vertex_maps[4][0].get_array())
gt.graph_draw(
    sub_G,
    vertex_text=sub_G.vp["title"],
    vertex_font_size=8,
    edge_pen_width=4,
    bg_color="white",
    output_size=(600, 600),
)

In [ ]:
root_v = gt.find_vertex(G_gt, G_gt.vp["id"], G_gt.gp["root"])[0]
dist_from_root = gt.shortest_distance(G_gt, root_v)

gt.graph_draw(
    G_gt,
    bg_color="white",
    vertex_fill_color=dist_from_root,
    output_size=(1200, 1200),
)